In [1]:
import sys
!{sys.executable} -m pip install pymongo
!{sys.executable} -m pip install pyyaml

In [2]:
# Get Mongo database
from yaml import load
from pymongo import MongoClient
from getsecret import getsecret

client = MongoClient(getsecret("MONGODB_URI"))
db = client[getsecret("DB_NAME")]
import urllib.request as req
import json
accounts = json.loads(req.urlopen("http://localhost:5000/synced_emails").read().decode("utf-8"))
# counter for figures
counter = 0

In [4]:
# Look through errors.
error_types = {"NV": {}}
for name in db.list_collection_names():
    if "error" in name:
        for error in db[name].find({"timestamp":{"$gt": 1534446003520}}):
            #print(name)
            #print(json.loads(error["error"]))
            print(error['error'])
            stack_trace = json.loads(str(error["error"])["stackTrace"]
            version = "version"
            if version not in error:
                version = "NV"
            else:
                version = error[version]
                if version not in error_types:
                    error_types[version] = {}
            errors_for_version = error_types[version]
            if stack_trace not in errors_for_version:
                errors_for_version[stack_trace] = 1
            else:
                errors_for_version[stack_trace] += 1
for version in error_types:
    print("VERSION: " + str(version))
    for stack in error_types[version]:
        print("STACKTRACE WITH FREQ " +str(error_types[version][stack]))
        print(stack)
        print("\n\n")

{"userID":"1923a3ff08c9d526c602aee2","name":"Error","nativeException":{},"message":"The application crashed because of an uncaught exception. You can look at \"stackTrace\" or \"nativeException\" for more detailed information about the exception.","stackTrace":"An uncaught Exception occurred on \"main\" thread.\njava.lang.RuntimeException: Error receiving broadcast Intent { act=android.intent.action.USER_PRESENT flg=0x24200010 } in com.tns.gen.android.content.BroadcastReceiver_AccessibilityService_73_56_@689b925\n\tat android.app.LoadedApk$ReceiverDispatcher$Args.lambda$-android_app_LoadedApk$ReceiverDispatcher$Args_52497(LoadedApk.java:1323)\n\tat android.app.-$Lambda$aS31cHIhRx41653CMnd4gZqshIQ.$m$7(Unknown Source:4)\n\tat android.app.-$Lambda$aS31cHIhRx41653CMnd4gZqshIQ.run(Unknown Source:39)\n\tat android.os.Handler.handleCallback(Handler.java:790)\n\tat android.os.Handler.dispatchMessage(Handler.java:99)\n\tat android.os.Looper.loop(Looper.java:164)\n\tat android.app.ActivityThrea

{"userID":"5f37a84b0b8cc78676e45cd3","name":"Error","nativeException":{},"message":"The application crashed because of an uncaught exception. You can look at \"stackTrace\" or \"nativeException\" for more detailed information about the exception.","stackTrace":"An uncaught Exception occurred on \"main\" thread.\njava.lang.RuntimeException: Error receiving broadcast Intent { act=android.intent.action.USER_PRESENT flg=0x24000010 } in com.tns.gen.android.content.BroadcastReceiver_AccessibilityService_63_56_@ba3132f\n\tat android.app.LoadedApk$ReceiverDispatcher$Args.run(LoadedApk.java:1175)\n\tat android.os.Handler.handleCallback(Handler.java:836)\n\tat android.os.Handler.dispatchMessage(Handler.java:103)\n\tat android.os.Looper.loop(Looper.java:203)\n\tat android.app.ActivityThread.main(ActivityThread.java:6361)\n\tat java.lang.reflect.Method.invoke(Native Method)\n\tat com.android.internal.os.ZygoteInit$MethodAndArgsCaller.run(ZygoteInit.java:1063)\n\tat com.android.internal.os.ZygoteIn

{"userID":"4355d7465db8d7e2b2534251","name":"Error","nativeException":{},"message":"The application crashed because of an uncaught exception. You can look at \"stackTrace\" or \"nativeException\" for more detailed information about the exception.","stackTrace":"An uncaught Exception occurred on \"main\" thread.\ncom.tns.NativeScriptException: \nCalling js method run failed\n\nTypeError: Cannot read property 'setOnTouchListener' of null\nFile: \"file:///data/data/com.stanfordhci.habitlab/files/app/tns_modules/tns-core-modules/ui/core/view/view.js, line: 213, column: 37\n\nStackTrace: \n\tFrame: function:'View.onUnloaded', file:'file:///data/data/com.stanfordhci.habitlab/files/app/tns_modules/tns-core-modules/ui/core/view/view.js', line: 213, column: 38\n\tFrame: function:'', file:'file:///data/data/com.stanfordhci.habitlab/files/app/tns_modules/tns-core-modules/ui/core/view-base/view-base.js', line: 315, column: 92\n\tFrame: function:'ViewBase.callFunctionWithSuper', file:'file:///data/

{"userID":"noIDFound","name":"Error","nativeException":{},"message":"The application crashed because of an uncaught exception. You can look at \"stackTrace\" or \"nativeException\" for more detailed information about the exception.","stackTrace":"An uncaught Exception occurred on \"main\" thread.\njava.lang.RuntimeException: Unable to start receiver com.habitlab.NewPackageReceiver: com.tns.NativeScriptException: \nCalling js method onReceive failed\n\nSyntaxError: Unexpected token u in JSON at position 0\nFile: \"<unknown>, line: 1, column: 0\n\nStackTrace: \n\tFrame: function:'exports.removePackage', file:'file:///data/data/com.stanfordhci.habitlab/files/app/util/StorageUtil.js', line: 442, column: 19\n\tFrame: function:'onReceive', file:'file:///data/data/com.stanfordhci.habitlab/files/app/receivers/NewPackageReceiver.js', line: 23, column: 19\n\n\n\tat android.app.ActivityThread.handleReceiver(ActivityThread.java:3039)\n\tat android.app.ActivityThread.-wrap18(ActivityThread.java)\n\

{"userID":"bc8004f2d5954bac43740d66","name":"Error","nativeException":{},"message":"The application crashed because of an uncaught exception. You can look at \"stackTrace\" or \"nativeException\" for more detailed information about the exception.","stackTrace":"An uncaught Exception occurred on \"main\" thread.\njava.lang.RuntimeException: Error receiving broadcast Intent { act=android.intent.action.USER_PRESENT flg=0x24200010 } in com.tns.gen.android.content.BroadcastReceiver_AccessibilityService_73_56_@3a40176\n\tat android.app.LoadedApk$ReceiverDispatcher$Args.lambda$-android_app_LoadedApk$ReceiverDispatcher$Args_52225(LoadedApk.java:1329)\n\tat android.app.-$Lambda$FilBqgnXJrN9Mgyks1XHeAxzSTk.$m$0(Unknown Source:4)\n\tat android.app.-$Lambda$FilBqgnXJrN9Mgyks1XHeAxzSTk.run(Unknown Source:0)\n\tat android.os.Handler.handleCallback(Handler.java:789)\n\tat android.os.Handler.dispatchMessage(Handler.java:98)\n\tat android.os.Looper.loop(Looper.java:164)\n\tat android.app.ActivityThread

...

{"userID":"U1532437453047928","name":"Error","nativeException":{},"message":"The application crashed because of an uncaught exception. You can look at \"stackTrace\" or \"nativeException\" for more detailed information about the exception.","stackTrace":"An uncaught Exception occurred on \"main\" thread.\njava.lang.RuntimeException: Error receiving broadcast Intent { act=android.intent.action.USER_PRESENT flg=0x24200010 } in com.tns.gen.android.content.BroadcastReceiver_AccessibilityService_73_56_@563f410\n\tat android.app.LoadedApk$ReceiverDispatcher$Args.lambda$getRunnable$0(LoadedApk.java:1401)\n\tat android.app.-$$Lambda$LoadedApk$ReceiverDispatcher$Args$_BumDX2UKsnxLVrE6UJsJZkotuA.run(Unknown Source:2)\n\tat android.os.Handler.handleCallback(Handler.java:873)\n\tat android.os.Handler.dispatchMessage(Handler.java:99)\n\tat android.os.Looper.loop(Looper.java:193)\n\tat android.app.ActivityThread.main(ActivityThread.java:6669)\n\tat java.lang.reflect.Method.invoke(Native Method)\n\ta

{"userID":"87cd8b98991c55a2a3c0e93f","name":"Error","nativeException":{},"message":"The application crashed because of an uncaught exception. You can look at \"stackTrace\" or \"nativeException\" for more detailed information about the exception.","stackTrace":"An uncaught Exception occurred on \"main\" thread.\njava.lang.RuntimeException: Error receiving broadcast Intent { act=android.intent.action.SCREEN_ON flg=0x58200010 } in com.tns.gen.android.content.BroadcastReceiver_AccessibilityService_63_56_@208c211\n\tat android.app.LoadedApk$ReceiverDispatcher$Args.lambda$-android_app_LoadedApk$ReceiverDispatcher$Args_53997(LoadedApk.java:1362)\n\tat android.app.-$Lambda$aS31cHIhRx41653CMnd4gZqshIQ.$m$7(Unknown Source:4)\n\tat android.app.-$Lambda$aS31cHIhRx41653CMnd4gZqshIQ.run(Unknown Source:39)\n\tat android.os.Handler.handleCallback(Handler.java:790)\n\tat android.os.Handler.dispatchMessage(Handler.java:99)\n\tat android.os.Looper.loop(Looper.java:164)\n\tat android.app.ActivityThread.m

{"userID":"421619f90f941a563deb6eb9","name":"Error","nativeException":{},"message":"The application crashed because of an uncaught exception. You can look at \"stackTrace\" or \"nativeException\" for more detailed information about the exception.","stackTrace":"An uncaught Exception occurred on \"main\" thread.\njava.lang.RuntimeException: Error receiving broadcast Intent { act=android.intent.action.USER_PRESENT flg=0x24200010 } in com.tns.gen.android.content.BroadcastReceiver_AccessibilityService_73_56_@6c62302\n\tat android.app.LoadedApk$ReceiverDispatcher$Args.lambda$-android_app_LoadedApk$ReceiverDispatcher$Args_52226(LoadedApk.java:1329)\n\tat android.app.-$Lambda$FilBqgnXJrN9Mgyks1XHeAxzSTk.$m$0(Unknown Source:4)\n\tat android.app.-$Lambda$FilBqgnXJrN9Mgyks1XHeAxzSTk.run(Unknown Source:0)\n\tat android.os.Handler.handleCallback(Handler.java:789)\n\tat android.os.Handler.dispatchMessage(Handler.java:98)\n\tat android.os.Looper.loop(Looper.java:164)\n\tat android.app.ActivityThread

{"userID":"d8ab95f53d6f885c6f5efc9c","name":"Error","nativeException":{},"message":"The application crashed because of an uncaught exception. You can look at \"stackTrace\" or \"nativeException\" for more detailed information about the exception.","stackTrace":"An uncaught Exception occurred on \"main\" thread.\ncom.tns.NativeScriptException: \nCalling js method onAccessibilityEvent failed\n\nReferenceError: name is not defined\nFile: \"file:///data/data/com.stanfordhci.habitlab/files/app/util/StorageUtil.js, line: 1571, column: 45\n\nStackTrace: \n\tFrame: function:'exports.isPackageFrequent', file:'file:///data/data/com.stanfordhci.habitlab/files/app/util/StorageUtil.js', line: 1571, column: 46\n\tFrame: function:'onAccessibilityEvent', file:'file:///data/data/com.stanfordhci.habitlab/files/app/services/AccessibilityService.js', line: 202, column: 26\n\n\n\tat com.tns.Runtime.callJSMethodNative(Native Method)\n\tat com.tns.Runtime.dispatchCallJSMethodNative(Runtime.java:1101)\n\tat c

TypeError: the JSON object must be str, bytes or bytearray, not 'dict'

In [4]:
# Unfortunately, we're still getting those erroneous logs of huge time spent on "" domain since some updates aren't fully reflected yet. Let's delete them again.
done = False
for account in accounts:
    if not done:
        for user in account["android"]:
            if not done:
                print("USER ---- " + user)
                if len([s for s in db[user + "_sessions"].find({"duration": {"$gt": 60 ** 2 * 24 * 7}})]) > 0:
                    print("Num of sessions: " + str(len([s for s in db[user+"_sessions"].find()])))
                    print("Num of bad sessions: " + str(len([s for s in db[user + "_sessions"].find({"domain": "", "duration": {"$gt": 60 ** 2 * 24}})])))
                    print("VERSIONS" + str({s['version'] for s in db[user + "_sessions"].find({"domain": "", "duration": {"$gt": 60 ** 2 * 24}})}))
                    db[user + "_sessions"].delete_many({"domain": "","duration": {"$gt": 60 ** 2 * 24}})
                    print("After delete: " + str(len([s for s in db[user+"_sessions"].find()])))



USER ---- 9cb8c59eabbb60ad6447345c
USER ---- b219926d16e6e46fa426cafd
Num of sessions: 695
Num of bad sessions: 9
VERSIONS{38}
After delete: 686
USER ---- U1532437453047928
Num of sessions: 2255
Num of bad sessions: 24
VERSIONS{38}
After delete: 2231
USER ---- 010cf6179f52b3d62184ee17
Num of sessions: 5353
Num of bad sessions: 122
VERSIONS{38}
After delete: 5231
USER ---- 0b0d1da5c036cb9637aff5d5
Num of sessions: 2635
Num of bad sessions: 82
VERSIONS{37}
After delete: 2553
USER ---- f6c5e87bbe828ffc4278d4b7
USER ---- 38a4b3e06ead18a13442f5a7
USER ---- f463f9d5191a4bc1d1374298
USER ---- c4f80e629327886902828290
USER ---- 7f2cc2756b7c8c658a86ce6c
USER ---- 6351db97a5f1304f5dd86f0f
USER ---- 619164f4d106d7c53658931c
USER ---- 640c8c27aad1006852979eb5
USER ---- 8efcdb3217ad19868c86c2a4
USER ---- bdc472cb840f151f72f88ec3
USER ---- ec0dbd76f4093d26176ba153
USER ---- 855de136c8e71443d24b081b
USER ---- c8dc7f9f2901e94fd90b9a05
USER ---- 3d9e01f13145754f3ee22cdb
Num of sessions: 4054
Num of bad

USER ---- bf8940e1b328a52d7762f360
USER ---- db2cf71fab292769f9344019
Num of sessions: 5143
Num of bad sessions: 207
VERSIONS{38}
After delete: 4936
USER ---- 42c85dc535a91bfddb10fa7f
Num of sessions: 3680
Num of bad sessions: 105
VERSIONS{28}
After delete: 3575
USER ---- 3d24f801df0c9b67efe94fcf
USER ---- 6bb9b0f3013ebe1e49c29fdb
USER ---- cbba1ef901015c7d3423cfb8
USER ---- 881a0d247c847afb93b228ca
USER ---- e8c4b0f1c4bdd3443014083e
USER ---- 9bf667d2003691cdf9985a8d
USER ---- 94c8712cd7d31509230841d4
USER ---- 14b55e9cf4cadcd3eb40657c
USER ---- bab02945c9756596a03659d4
Num of sessions: 11601
Num of bad sessions: 215
VERSIONS{38}
After delete: 11386
USER ---- 23cab28916f19d8853ad97a2
Num of sessions: 6426
Num of bad sessions: 72
VERSIONS{38}
After delete: 6354
USER ---- c2f1a983aeac2e8c0efd2b44
USER ---- c5767a67d7df98e85a87742e
USER ---- 7c75881d2928009bb4c8982f
USER ---- b22f0cd5b3ca81a84192f47c
USER ---- b8cd3ebbc5d2eb8419df5ad6
USER ---- 2769eca36b0c57c6a786d4c0
USER ---- 64279a45

Num of sessions: 1895
Num of bad sessions: 36
VERSIONS{38}
After delete: 1859
USER ---- e2dee48c11ee8fa6b022e27d
USER ---- 9dc8d75e4ea26a4d453a0a4e
USER ---- 2ec0a290278ccb45a15ad811
USER ---- f515b2869f85570284766b2f
Num of sessions: 1286
Num of bad sessions: 84
VERSIONS{38}
After delete: 1202
USER ---- 8fed16ab78f1fbc0a5970db7
Num of sessions: 1243
Num of bad sessions: 32
VERSIONS{38}
After delete: 1211
USER ---- 9ded30824edd198c502a8c4d
USER ---- b1446158422d5289512ee210
USER ---- ed5a4131dbcfb7414e934b8a
USER ---- fcfe2d236ef3bc4734f15d3c
Num of sessions: 1794
Num of bad sessions: 14
VERSIONS{38}
After delete: 1780
USER ---- 59548965c2b36a57e653acc3
Num of sessions: 1453
Num of bad sessions: 73
VERSIONS{38}
After delete: 1380
USER ---- 531796cd1d22b33882a7ce7e
USER ---- 480fd7372d25f00764c413be
USER ---- 0257fce54390565ba9046e8c
USER ---- 61a9a2958daf98fe099e1413
USER ---- 96d43a5103e829349dd69273
Num of sessions: 1205
Num of bad sessions: 240
VERSIONS{37}
After delete: 965
USER --